# NLP - HW5
### Miguel Bonilla

In [1]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

- [1. Compile a List of Reviews](#1.-Compile-a-List-of-Reviews)
- [2. Extract Noun Phrase Chunks](#2.-Extract-Noun-Phrase-Chunks)
- [3. Output All the Chunks](#3.-Output-All-the-Chunks)

### 1. Compile a List of Reviews

Using BeautifulSoup to crawl through the featured 25 reviews for each of the films, for a total of 100 reviews. The four movies are: "The Thing (1982)", "A Quiet Place", "Alien Covenant", and "The Shining". Each of these has a mix of positive and negative reviews, though the distribution of these depends on the 'quality' of the film. 

In [2]:
### assign headers since IMDB rejects the requests without it
headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.50'}

In [3]:
## The Thing (1982)
url1 = 'https://www.imdb.com/title/tt0084787/reviews/?ref_=tt_ov_rt'
## A Quiet Place
url2 = 'https://www.imdb.com/title/tt6644200/reviews/?ref_=adv_li_tt'
## Alien Covenant
url3 = 'https://www.imdb.com/title/tt2316204/reviews/?ref_=adv_li_tt'
## The Shining
url4 = 'https://www.imdb.com/title/tt0081505/reviews/?ref_=fn_al_tt_1'

#movie list
movies = ['The_Thing','A_Quiet_Place','Alien_Covenant','The_Shining']
#url list
urls = [url1, url2, url3, url4]

In [25]:
def get_links(movielist,urllist):
    refs = []
    title = []
    index = []
    for i in range(len(movielist)):
        movie = get(urllist[i],headers=headers)
        movie_soup = BeautifulSoup(movie.content,'html.parser')
        container = movie_soup.find_all(class_='title')
        for j in range(len(container)):
            refs.append('https://www.imdb.com'+ container[j]['href'])
            title.append(movielist[i])
            index.append('{}_{}'.format(movielist[i],j))
    return(pd.DataFrame({'movie':title,
                         'url':refs,
                         'review':index
                        }))

In [26]:
review_links = get_links(movies,urls)
review_links

,movie,url,review
0,The_Thing,https://www.imdb.com/review/rw0197822/?ref_=tt...,The_Thing_0
1,The_Thing,https://www.imdb.com/review/rw3346521/?ref_=tt...,The_Thing_1
2,The_Thing,https://www.imdb.com/review/rw1833451/?ref_=tt...,The_Thing_2
3,The_Thing,https://www.imdb.com/review/rw6379386/?ref_=tt...,The_Thing_3
4,The_Thing,https://www.imdb.com/review/rw0197779/?ref_=tt...,The_Thing_4
...,...,...,...
95,The_Shining,https://www.imdb.com/review/rw0179869/?ref_=tt...,The_Shining_20
96,The_Shining,https://www.imdb.com/review/rw7669173/?ref_=tt...,The_Shining_21
97,The_Shining,https://www.imdb.com/review/rw0180168/?ref_=tt...,The_Shining_22
98,The_Shining,https://www.imdb.com/review/rw2504629/?ref_=tt...,The_Shining_23


In [6]:
review_links.url[1]

'https://www.imdb.com/review/rw3346521/?ref_=tt_urv'

### 2. Extract Noun Phrase Chunks

#### a. Grab the review text from each link

In [27]:
def grab_review(links_table):
    tokens = []
    for i in range(len(links_table)):
        review = get(links_table.url[i],headers)
        review_soup = BeautifulSoup(review.content, 'html.parser')
        sent = []
        for string in review_soup.find(class_='text show-more__control').stripped_strings:
            sent.append(sent_tokenize(string))
        tokens.append([item for sublist in sent for item in sublist])
    return(pd.DataFrame({'movie':links_table.movie,
                         'tokens':tokens,
                         'review':links_table.review
                        }))

In [28]:
tokens2 = grab_review(review_links)

In [29]:
tokens2

,movie,tokens,review
0,The_Thing,"[""I know I'm human., And if you were all these...",The_Thing_0
1,The_Thing,"[A classic film., John Carpenter's ""The Thing""...",The_Thing_1
2,The_Thing,[John Carpenter shows how much he loves the 19...,The_Thing_2
3,The_Thing,"[""The ultimate in alien terror,"" it says., It'...",The_Thing_3
4,The_Thing,"[Remake of the classic 1951 ""The Thing From An...",The_Thing_4
...,...,...,...
95,The_Shining,"[Chilling, majestic piece of cinematic fright,...",The_Shining_20
96,The_Shining,[The Shining is directed by Stanley Kubrick an...,The_Shining_21
97,The_Shining,"[*!, !- SPOILERS - !, !, *, Before I begin thi...",The_Shining_22
98,The_Shining,[This film is currently the 48th highest rated...,The_Shining_23


### 3. Output All the Chunks

In [14]:
print('hi my name is {}'.format('Miguel'))

hi my name is Miguel


In [17]:
print('{}_{}'.format('Miguel','Bonilla'))

Miguel_Bonilla
